In [5]:
!pip install git+https://github.com/ageitgey/face_recognition_models


  Cloning https://github.com/ageitgey/face_recognition_models to c:\users\bit\appdata\local\temp\pip-req-build-xoozf7q7
  Resolved https://github.com/ageitgey/face_recognition_models to commit e67de717267507d1e9246de95692eb8be736ab61
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for face_recognition_models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566223 sha256=28df7525f34bd78a6e78937214a8f322e77ca00d07d4e72f0382814a2a09569e
  Stored in directory: C:\Users\BIT\AppData\Local\Temp\pip-ephem-wheel-cache-4mrww8sz\wheels\6e\e8\c4\d1903c1699af945f6e2c9933775055d32a71c5078ba9819957
Successfully built face_recognition_models


  Running command git clone --filter=blob:none --quiet https://github.com/ageitgey/face_recognition_models 'C:\Users\BIT\AppData\Local\Temp\pip-req-build-xoozf7q7'
  DEPRECATION: Building 'face_recognition_models' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'face_recognition_models'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [6]:
import face_recognition
import cv2
import numpy as np
import csv
from datetime import datetime

In [12]:
import face_recognition
import cv2
import numpy as np
import csv
from datetime import datetime

# Start webcam
video_capture = cv2.VideoCapture(0)

# Load known face encodings
harry_image = face_recognition.load_image_file("faces/harry.jpeg")
harry_encoding = face_recognition.face_encodings(harry_image)[0]

larry_image = face_recognition.load_image_file("faces/larry.png")
larry_encoding = face_recognition.face_encodings(larry_image)[0]

known_face_encodings = [harry_encoding, larry_encoding]
known_face_names = ["Harry", "Larry"]

# Track who has been marked present
students = known_face_names.copy()

# Create CSV file with today's date
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
f = open(f"{current_date}.csv", "w+", newline="")
lnwriter = csv.writer(f)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Find face locations and encodings
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distance = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distance)
        name = "Unknown"

        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw label and box on original frame (not resized)
        top, right, bottom, left = face_location
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name + " Present", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

        # Write attendance to CSV if student is recognized and not yet marked
        if name in students:
            students.remove(name)
            now = datetime.now()  # update timestamp
            current_time = now.strftime("%H:%M:%S")
            lnwriter.writerow([name, current_time])

    # Display the frame
    cv2.imshow("Attendance", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
video_capture.release()
cv2.destroyAllWindows()
f.close()
